In [2]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np

# Load BERT tokenizer and model from Hugging Face

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

# Example emotion-labeled dataset (text, label)

In [ ]:
texts = [
    "I am so happy today!",         # joy
    "This is absolutely terrible",  # anger
    "I'm feeling really down",      # sadness
    "You did a great job!",         # joy
    "Why would you say that?" ,     # anger
    "How dare you!"                 # anger
]
labels = [0, 1, 2, 0, 1,1]  # 0 = joy, 1 = anger, 2 = sadness

# Tokenize input texts

In [15]:
encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='tf')

# Get BERT embeddings (pooled_output is [CLS] token representation)

In [16]:
outputs = bert_model(encodings)
pooled_output = outputs.pooler_output  # Shape: (batch_size, 768)

# Build emotion classifier model on top of BERT embeddings

In [17]:
classifier = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(768,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 emotion classes
])

# Compile and train

In [18]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.fit(pooled_output, np.array(labels), epochs=20, verbose=0)

# Inference on new text

In [19]:
def classify_emotion(text):
    inputs = tokenizer(text, return_tensors='tf', padding=True, truncation=True)
    outputs = bert_model(inputs)
    pooled = outputs.pooler_output
    pred = classifier.predict(pooled)[0]
    emotion = ["Joy", "Anger", "Sadness"][np.argmax(pred)]
    return emotion

# Test the classifier

In [22]:
test_text = "hoWW dare you say that!"
print("Text:", test_text)
print("Predicted Emotion:", classify_emotion(test_text))

Text: hoWW dare you say that!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Predicted Emotion: Anger
